In [1]:
import cv2
import numpy as np 
import pandas as pd 
from glob import glob
import os
from copy import copy
from PIL import Image
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import tensorflow.keras as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import datasets, layers, models, losses, Model
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

2023-07-10 11:05:10.126292: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 11:05:10.252211: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-10 11:05:10.722760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/usr/local/cuda-11.6/lib64:/home/gil/anaconda3/envs/LeeYS/lib/
2023-07-10 11:05:10.722855: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library

In [2]:
img_list=glob('../../data/360umTile/*.jpg')
img_slidename_list=copy(img_list)
for i in range(len(img_list)):
    img_slidename_list[i]=os.path.basename(img_list[i])[:9]
label=pd.read_csv('../../data/P53&MMR.csv')
img_index_list=[]
for j in range(len(label['PathologyNumber'])):
    img_index_list.append([i for i, ele in enumerate(img_slidename_list) if ele == label['PathologyNumber'][j]])
label_list=copy(label['MMR status'].to_list())
for i in range(len(label_list)):
    if label_list[i]=="Defect":
        label_list[i]=1
    else:
        label_list[i]=0

In [3]:
x_train,x_test,y_train,y_test=train_test_split(img_index_list,label_list, test_size=0.1, random_state=42)

In [4]:
list_x_train,list_x_test,list_y_train,list_y_test=[],[],[],[]
for i in range(len(x_train)):
    x_list=[]
    if len(x_train[i])>100:                                  
        for k in range(100):
            a = random.randint(x_train[i][0],x_train[i][len(x_train[i])-1])       
            while a in x_list :              # a가 이미 뽑은 리스트에 있을 때까지 다시 뽑자
                a = random.randint(x_train[i][0],x_train[i][len(x_train[i])-1])
            x_list.append(a) # 새로운 a 값을 리스트에 추가
    else:
        x_list=copy(x_train[i])
    for j in x_list:
        list_x_train.append(img_list[j])
        list_y_train.append(y_train[i])
        
for i in range(len(x_test)):
    x_list=[]
    if len(x_test[i])>100:                                  
        for k in range(100):
            a = random.randint(x_test[i][0],x_test[i][len(x_test[i])-1])       
            while a in x_list :              # a가 이미 뽑은 리스트에 있을 때까지 다시 뽑자
                a = random.randint(x_test[i][0],x_test[i][len(x_test[i])-1])
            x_list.append(a) # 새로운 a 값을 리스트에 추가
    else:
        x_list=copy(x_test[i])
    for j in x_list:
        list_x_test.append(img_list[j])
        list_y_test.append(y_test[i])
    

In [5]:
img_size=224
x_train=np.zeros((len(list_x_train),img_size,img_size,3))
x_test=np.zeros((len(list_x_test),img_size,img_size,3))
y_train=np.zeros((len(list_y_train),1),dtype=np.int16)
y_test=np.zeros((len(list_y_test),1),dtype=np.int16)

for i in range(len(list_x_train)):
    x_train[i]=Image.open(list_x_train[i])
    y_train[i]=list_y_train[i]

for i in range(len(list_x_test)):
    x_test[i]=Image.open(list_x_test[i])
    y_test[i]=list_y_test[i]

In [6]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-8)
earlystopper = EarlyStopping(monitor='val_loss',patience=40, verbose=1)
check_point = K.callbacks.ModelCheckpoint(filepath="../../model/Resnet50_call.h5",
                                              monitor='val_loss',
                                              mode="min",
                                              save_best_only=True
                                              )
input_t=K.Input(shape=(img_size, img_size, 3))
input_tensor = layers.experimental.preprocessing.Resizing(img_size, img_size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
ResNet=ResNet50(include_top=True,weights=None,input_tensor=input_tensor)
model_ResNet = K.models.Sequential()
model_ResNet.add(ResNet)
model_ResNet.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model_ResNet.add(K.layers.Dense(1, activation='sigmoid'))
model_ResNet.compile(optimizer=K.optimizers.Adam(lr=2e-3),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
hist_ResNet = model_ResNet.fit(x_train, y_train,callbacks=[check_point,reduce_lr,earlystopper],validation_data=(x_test, y_test), epochs=300, batch_size=32,shuffle=True)
model_ResNet.save("../../model/Resnet50.h5")

2023-07-10 11:19:53.551251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 11:19:54.065796: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38196 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/300


2023-07-10 11:20:13.742141: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-07-10 11:20:14.622217: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


905/905 [==============================] - 68s 70ms/step - loss: 0.5168 - accuracy: 0.7906 - val_loss: 0.5454 - val_accuracy: 0.7683 - lr: 0.0020
Epoch 2/300
905/905 [==============================] - 63s 69ms/step - loss: 0.5139 - accuracy: 0.7906 - val_loss: 0.5448 - val_accuracy: 0.7683 - lr: 0.0020
Epoch 3/300
905/905 [==============================] - 63s 69ms/step - loss: 0.5138 - accuracy: 0.7906 - val_loss: 0.5442 - val_accuracy: 0.7683 - lr: 0.0020
Epoch 4/300
905/905 [==============================] - 62s 69ms/step - loss: 0.5136 - accuracy: 0.7906 - val_loss: 0.5413 - val_accuracy: 0.7683 - lr: 0.0020
Epoch 5/300
905/905 [==============================] - 59s 66ms/step - loss: 0.5140 - accuracy: 0.7906 - val_loss: 0.5441 - val_accuracy: 0.7683 - lr: 0.0020
Epoch 6/300
905/905 [==============================] - 60s 66ms/step - loss: 0.5139 - accuracy: 0.7906 - val_loss: 0.5433 - val_accuracy: 0.7683 - lr: 0.0020
Epoch 7/300
905/905 [==============================] - 61s 67ms/

KeyboardInterrupt: 